# **Tugas - 3 Modelling (Logistic Regression) dan Testing**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pickle
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/My Drive/PPW-A/report/Tugas-PPW/hasil_preprocesing.csv")
df.head()

,judul,isi,tanggal,kategori,Hasil cleansing,Hasil case_folding,tokenize,Hasil stopword
0,Game Battle of Guardian Gelar Eksibisi di PON ...,Jakarta - Pada ajang Pekan Olahraga Nasional (...,"Minggu, 08 Sep 2024 19:03 WIB",Games,Jakarta Pada ajang Pekan Olahraga Nasional PO...,jakarta pada ajang pekan olahraga nasional po...,"['jakarta', 'pada', 'ajang', 'pekan', 'olahrag...",jakarta ajang pekan olahraga nasional pon aceh...
1,"Setelah Viral, Game Hamster Kombat akan Jadi T...","Jakarta - Bitget, exchange mata uang kripto, m...","Minggu, 08 Sep 2024 15:03 WIB",Games,Jakarta Bitget exchange mata uang kripto meng...,jakarta bitget exchange mata uang kripto meng...,"['jakarta', 'bitget', 'exchange', 'mata', 'uan...",jakarta bitget exchange mata uang kripto mengu...
2,3 Cara Log Out Akun ML 2024 dan Beralih Akun Baru,Jakarta - Mobile Legends (ML) adalah salah sat...,"Sabtu, 07 Sep 2024 19:41 WIB",Games,Jakarta Mobile Legends ML adalah salah satu g...,jakarta mobile legends ml adalah salah satu g...,"['jakarta', 'mobile', 'legends', 'ml', 'adalah...",jakarta mobile legends ml salah game online po...
3,Ada Apa dengan Indostars di FFWS SEA 2024 Fall...,Jakarta - Free Fire World Series (FFWS) Southe...,"Sabtu, 07 Sep 2024 17:18 WIB",Games,Jakarta Free Fire World Series FFWS Southeeas...,jakarta free fire world series ffws southeeas...,"['jakarta', 'free', 'fire', 'world', 'series',...",jakarta free fire world series ffws southeeast...
4,"NBA 2K25 Rilis di PS4, PS5, Xbox, dan PC, Segi...",Jakarta - 2K selaku penerbit dan Visual Concep...,"Sabtu, 07 Sep 2024 16:14 WIB",Games,Jakarta K selaku penerbit dan Visual Concept ...,jakarta k selaku penerbit dan visual concept ...,"['jakarta', 'k', 'selaku', 'penerbit', 'dan', ...",jakarta k penerbit visual concept pengembang g...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Menginisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menghitung TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['Hasil stopword'])

# Mengubah hasilnya menjadi DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.head(10)
# Menyimpan hasil ke file CSV
tfidf_df.to_csv('hasil_tfidf.csv', index=False)

In [ ]:
# Menggunakan kolom 'cleaned_text' sebagai teks input dan 'kategori' sebagai label
texts = df['Hasil stopword'].fillna('')  # Mengganti NaN dengan string kosong jika ada
labels = df['kategori'].fillna('unknown')  # Mengganti NaN di kategori jika ada

# Melihat label yang digunakan untuk klasifikasi
print(labels.unique())

['Games' 'Sepak Bola']


In [ ]:
from sklearn.model_selection import train_test_split
# Melakukan transformasi TF-IDF
tfidf = TfidfVectorizer(max_features=None)  # Menggunakan 5000 fitur teratas
X_tfidf = tfidf.fit_transform(texts)

# Memisahkan dataset menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)

# Melihat bentuk dari hasil TF-IDF
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of X_train: (80, 5288)
Shape of X_test: (20, 5288)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Membuat model Logistic Regression dengan regularisasi
model = LogisticRegression(max_iter=1000, C=0.3)  # Mengurangi nilai C untuk menambah regularisasi

# Melatih model dengan data yang sudah diresample (jika diperlukan)
model.fit(X_train, y_train)

# Memprediksi hasil pada data testing
y_pred = model.predict(X_test)

In [ ]:
# Evaluasi hasil prediksi
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Menampilkan laporan klasifikasi
print(classification_report(y_test, y_pred))

Accuracy: 90.00%
              precision    recall  f1-score   support

       Games       1.00      0.83      0.91        12
  Sepak Bola       0.80      1.00      0.89         8

    accuracy                           0.90        20
   macro avg       0.90      0.92      0.90        20
weighted avg       0.92      0.90      0.90        20



In [ ]:
# Menyimpan tfidf_matrix dan vectorizer menggunakan pickle
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)
# Menyimpan model ke dalam file pickle
with open('logistic_regression_model.pkl', 'wb') as file:
    pickle.dump(model, file)

**PENGUJIAN**

---

In [ ]:
import joblib

# Memuat vectorizer dan model
tfidf = joblib.load('tfidf_vectorizer.pkl')
model = joblib.load('logistic_regression_model.pkl')

In [ ]:
# Teks baru yang ingin diprediksi
new_text = ['''Jakarta - Dalam ranking FIFA terbaru, Jepang masih perkasa sebagai tim Asia dengan peringkat tertinggi. Pasukan Samurai Biru ada di peringkat ke-16!
FIFA merilis ranking FIFA terbaru per September 2024 pada Jumat (19/9). Untuk di level Asia, Jepang masih menempati peringkat teratas.
Timnas Jepang duduk di peringkat ke-16 ranking FIFA terbaru dengan 1639.6 poin. Wataru Endo dkk naik dua peringkat.
epang mampu sapu bersih dua kemenangan di dua laga awal ronde ketiga Kualifikasi Piala Dunia 2026 Zona Asia. Jepang pesta gol atas China 7-0 dan cukur Bahrain 5-0.
Jepang pun menjadi satu-satunya tim dengan poin sempurna sejauh ini di Grup C dengan enam poin. Mereka disusul Arab Saudi (empat poin), Bahrain (tiga poin), dan Indonesia (dua poin).''']

# Preprocessing dan transformasi menggunakan TF-IDF yang sudah disimpan
new_text_tfidf = tfidf.transform(new_text)

# Melakukan prediksi menggunakan model Logistic Regression
prediction = model.predict(new_text_tfidf)

# Hasil prediksi
predicted_category = "Games" if prediction[0] == 'Games' else "Sepak Bola"
print(f"Prediksi: {predicted_category}")

Prediksi: Sepak Bola


Test 1

Jakarta - Dalam ranking FIFA terbaru, Jepang masih perkasa sebagai tim Asia dengan peringkat tertinggi. Pasukan Samurai Biru ada di peringkat ke-16!
FIFA merilis ranking FIFA terbaru per September 2024 pada Jumat (19/9). Untuk di level Asia, Jepang masih menempati peringkat teratas.
Timnas Jepang duduk di peringkat ke-16 ranking FIFA terbaru dengan 1639.6 poin. Wataru Endo dkk naik dua peringkat.
epang mampu sapu bersih dua kemenangan di dua laga awal ronde ketiga Kualifikasi Piala Dunia 2026 Zona Asia. Jepang pesta gol atas China 7-0 dan cukur Bahrain 5-0.
Jepang pun menjadi satu-satunya tim dengan poin sempurna sejauh ini di Grup C dengan enam poin. Mereka disusul Arab Saudi (empat poin), Bahrain (tiga poin), dan Indonesia (dua poin).

Test 2

Jakarta - Sekitar satu dekade lalu internet terobsesi dengan game mobile Flappy Bird yang bikin ketagihan. Setelah menghilang cukup lama, game viral ini akan kembali lagi dengan membawa sejumlah pembaruan.
Reinkarnasi Flappy Bird dipelopori oleh sekumpulan penggemar yang tergabung dalam Flappy Bird Foundation. Mereka berhasil memperoleh hak cipta atas game ini dari Gametech, LLC, yang tidak memiliki hubungan dengan pencipta asli Flappy Bird, Dong Nguyen.
Game Flappy Bird akan tersedia untuk Android dan iOS mulai tahun 2025,
 tapi game ini akan bisa dimainkan di platform lain sebelum tanggal tersebut.
Tim Flappy Bird Foundation berencana menghadirkan game ini ke platform seperti desktop dan mobile web mulai musim gugur tahun ini.